# Nyström Kernel Approximation
Python implementation of the Nyström Kernel Approximation from https://arxiv.org/abs/1605.07583 and https://github.com/cnmusco/recursive-nystrom . Hopefully for later use with our work on kernel machines.

In [1]:
pwd

'/Users/cdickens/Documents/GitHub/phd_work'

In [2]:
import os
import tarfile
from six.moves import urllib

"""
DOWNLOAD_ROOT = "https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.data.gz"
COVER_PATH = os.path.join("datasets", "covertype")
COVER_URL = DOWNLOAD_ROOT + "datasets/covertype/cover.gz"

def fetch_data(data_url=COVER_URL, data_path=COVER_PATH):
    if not os.path.isdir(data_path):
        os.makedirs(data_path)
    tgz_path = os.path.join(data_path, "cover.gz")
    urllib.request.urlretrieve(data_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=data_path)
    housing_tgz.close()
"""

'\nDOWNLOAD_ROOT = "https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.data.gz"\nCOVER_PATH = os.path.join("datasets", "covertype")\nCOVER_URL = DOWNLOAD_ROOT + "datasets/covertype/cover.gz"\n\ndef fetch_data(data_url=COVER_URL, data_path=COVER_PATH):\n    if not os.path.isdir(data_path):\n        os.makedirs(data_path)\n    tgz_path = os.path.join(data_path, "cover.gz")\n    urllib.request.urlretrieve(data_url, tgz_path)\n    housing_tgz = tarfile.open(tgz_path)\n    housing_tgz.extractall(path=data_path)\n    housing_tgz.close()\n'

In [3]:
import pandas as pd
import numpy as np
import random

random.seed(1)

cov_data = pd.read_csv('datasets/covtype.data', header=None)

In [4]:
cov_data.shape
type(cov_data)

pandas.core.frame.DataFrame

In [5]:
# Choose a small subset for experiments
nsamples = 6000 
# remove the last column of labels

cov_data_shuffled = cov_data.iloc[np.random.permutation(len(cov_data))]
cov_data_shuffled.shape
cov_data_shuffled.head()

,0,1,2,3,4,5,6,7,8,9,...,45,46,47,48,49,50,51,52,53,54
263205,2234,309,25,283,94,760,144,214,207,437,...,0,0,0,0,0,0,0,0,0,3
227964,2990,124,11,430,88,1801,238,232,124,2932,...,1,0,0,0,0,0,0,0,0,2
227696,2084,238,33,124,82,240,144,246,224,95,...,0,0,0,0,0,0,0,0,0,3
23131,2845,130,30,95,42,4826,252,206,58,4689,...,0,0,0,0,0,0,0,0,0,2
160744,3253,50,4,841,109,6400,221,231,146,1565,...,0,0,0,0,0,0,0,0,0,2


In [6]:
X = cov_data_shuffled.iloc[0:nsamples].copy()

In [7]:
X.shape

(6000, 55)

In [8]:
X = X.drop(labels=54, axis=1)
#df = df.drop(labels='column_to_delete', axis=1)

In [9]:
X.shape
#X.iloc[:,0:10]

(6000, 54)

In [45]:
n, d = X.shape
means = np.zeros((1,d))
means = np.mean(X.iloc[:,:], axis=0)

In [25]:
means[means.index >= 10] = 0

In [26]:
means.shape

(54,)

In [27]:
means

0     2957.433333
1      156.088500
2       14.159167
3      267.284500
4       47.098000
5     2358.879833
6      212.266500
7      222.965333
8      142.076167
9     1996.079667
10       0.000000
11       0.000000
12       0.000000
13       0.000000
14       0.000000
15       0.000000
16       0.000000
17       0.000000
18       0.000000
19       0.000000
20       0.000000
21       0.000000
22       0.000000
23       0.000000
24       0.000000
25       0.000000
26       0.000000
27       0.000000
28       0.000000
29       0.000000
30       0.000000
31       0.000000
32       0.000000
33       0.000000
34       0.000000
35       0.000000
36       0.000000
37       0.000000
38       0.000000
39       0.000000
40       0.000000
41       0.000000
42       0.000000
43       0.000000
44       0.000000
45       0.000000
46       0.000000
47       0.000000
48       0.000000
49       0.000000
50       0.000000
51       0.000000
52       0.000000
53       0.000000
dtype: float64

In [35]:
X = X - np.tile(means, (nsamples, 1))

In [36]:
np.tile(means, (nsamples, 1))[0,:]

array([ 2957.43333333,   156.0885    ,    14.15916667,   267.2845    ,
          47.098     ,  2358.87983333,   212.2665    ,   222.96533333,
         142.07616667,  1996.07966667,     0.        ,     0.        ,
           0.        ,     0.        ,     0.        ,     0.        ,
           0.        ,     0.        ,     0.        ,     0.        ,
           0.        ,     0.        ,     0.        ,     0.        ,
           0.        ,     0.        ,     0.        ,     0.        ,
           0.        ,     0.        ,     0.        ,     0.        ,
           0.        ,     0.        ,     0.        ,     0.        ,
           0.        ,     0.        ,     0.        ,     0.        ,
           0.        ,     0.        ,     0.        ,     0.        ,
           0.        ,     0.        ,     0.        ,     0.        ,
           0.        ,     0.        ,     0.        ,     0.        ,
           0.        ,     0.        ])

In [43]:
X.iloc[0,:] # just to check that the correct subtractions are made

0    -6638.3000
1     -159.2655
2      -17.4775
3     -518.8535
4      -47.2940
5    -6316.6395
6     -492.7995
7     -454.8960
8     -219.2285
9    -5551.2390
10      -0.4550
11       0.0000
12       0.0000
13       1.0000
14       0.0000
15       0.0000
16       0.0000
17       0.0000
18       0.0000
19       0.0000
20       0.0000
21       0.0000
22       0.0000
23       1.0000
24       0.0000
25       0.0000
26       0.0000
27       0.0000
28       0.0000
29       0.0000
30       0.0000
31       0.0000
32       0.0000
33       0.0000
34       0.0000
35       0.0000
36       0.0000
37       0.0000
38       0.0000
39       0.0000
40       0.0000
41       0.0000
42       0.0000
43       0.0000
44       0.0000
45       0.0000
46       0.0000
47       0.0000
48       0.0000
49       0.0000
50       0.0000
51       0.0000
52       0.0000
53       0.0000
Name: 263205, dtype: float64

In [14]:
X.shape

(6000, 54)

In [60]:
new_vals = np.sum(X.iloc[:,0:10]**2,axis=0)
new_vals = new_vals.reshape((-1,1))
new_vals.shape

/Users/cdickens/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  


(10, 1)

In [72]:
col_norms = nsamples**(-0.5)*np.ones((1,d))
new_vals = 1/np.sqrt(np.sum(X.iloc[:,0:10]**2,axis=0))
new_vals = new_vals.values.reshape((1,-1))
#col_norms.shape
col_norms[:,0:10] = new_vals
col_norms[np.where(col_norms == np.Inf)] = 0

In [79]:
X = np.dot(X,np.diag(col_norms[0]))

In [82]:
# Set kernel parameters and construct explicit kernel
import scipy
from scipy.spatial.distance import pdist, squareform
gamma = 256
  # this is an NxD matrix, where N is number of items and D its dimensionalites
pairwise_sq_dists = squareform(pdist(X, 'sqeuclidean'))
K = scipy.exp(-256*pairwise_sq_dists)

In [83]:
K.shape

(6000, 6000)